## Measuring True Privacy Leakage

### Simple exercices
### Further exercices

#### Optimize `get_reference_attack`

Optimize the method `get_reference_attack` to only calculate one `out_models` for
every `example_to_attack` of the test cases.
This is possible because the `out_models` are trained using all training cases except
the one from the `examples_to_attack`.
And because all test cases will have the same `out_models`, it's enough to calculate this
model only once.

### Advanced exercices

In [ ]:
%run ml_load_data.ipynb
verbose_reference = False

In [ ]:
def get_reference_attack(train_model, X_train, y_train, X_test, y_test, 
                         num_examples_to_attack = 50, num_ref_models = 10):
    
    # Collect some arbitrary target examples to attack.
    examples_to_attack = []

    # ...half from the training data.
    for index in X_train.index[:num_examples_to_attack // 2]:
        examples_to_attack.append((index, X_train.loc[index], y_train.loc[index], 1))

    # ...half from the test data.
    for index in X_test.index[:num_examples_to_attack // 2]:
        examples_to_attack.append((index, X_test.loc[index], y_test.loc[index], 0))

    result = []
    target_model = train_model(X_train, y_train)

    # Now run the re-training attacks!
    for index, x, y, is_member in tqdm(examples_to_attack):
        # First, train a bunch of models without the target example 
        # (if it is in fact part of the training data)
        out_models = []
        for seed in range(num_ref_models):
            ref_model = train_model(
                X_train.drop(index=[index], errors="ignore"),
                y_train.drop(index=[index], errors="ignore"),
                seed=seed
            )
            out_models.append(ref_model)

        # Compute the attack features.
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            pred_in = target_model.predict_proba([x])
            preds_out = [model.predict_proba([x])[0] for model in out_models]

        logit_in = logit_scale(y, pred_in)
        logits_out = logit_scale(y, preds_out)

        # Next, we run a parametric test. We assume that "out" logits are 
        # Gaussian-distributed, so compute their mean and variance.
        logits_out_mean = np.mean(logits_out)
        logits_out_var = np.var(logits_out)

        # The parametric test is computing the probability that the "out" logits are 
        # less than "in" logit, which means that we predict the target record as a member:
        # 
        #   Pr[logit_out <= logit_in], where logit_out ~ Normal(mean, var) with mean and var
        #   estimated from reference models.
        #
        # See https://arxiv.org/abs/2112.03570, Eq. (4)
        prob = stats.norm(logits_out_mean, logits_out_var).cdf(logit_in) 

        result.append(dict(
            target_index=index,
            is_member=is_member,
            prob=prob,
        ))
        
    return result

In [ ]:
def plot_reference_attack(train_model, X_train, y_train, X_test, y_test, num_examples = 20):
    reference_attack = get_reference_attack(train_model, X_train, y_train, X_test, y_test, num_examples)
    plot_rfc_auroc(pd.DataFrame(reference_attack).is_member, pd.DataFrame(reference_attack).prob,
                  "Reference attack leakage")

if verbose_reference:
    plot_reference_attack(train_model, X_train, y_train, X_test, y_test)